# 환경설정 
환경설정을 잘못할경우 잘안되는경우가 많으니 꼭 확인할 것
실험후 버전 업데이트 하게 될경우 api에서 test할때 디펜던시 오류나는지 꼭 체크해보기

In [ ]:
#!pip install -r requirements.txt

# import

In [ ]:
#------------------------------------------------*  tool
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

#------------------------------------------------*  tensor, keras model

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.layers.core import Dense, Flatten
from keras.layers import Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau
#------------------------------------------------* sklearn, score, metrics
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn import metrics

#------------------------------------------------*  warnings filter


import warnings
warnings.filterwarnings(action="ignore")
#--------------------

import random

# Configuration
- 다른실험할때는 Class_name을 변경

In [ ]:
DATASET_PATH = "/104data/dataset/음성/voice_to_image/dataset20250127/"
# for문으로 처리하려고 했으나 각 진단마다 에폭이나 learning rate를 바꿔야 하는일들이 있어서 4개 타입 바꿔가면서 4번 돌릴것.
CLASS_NAME="normal_abnormal" #MCI_AD, SCI_AD, SCI_MCI,normal_abnormal
private_dir = f"{DATASET_PATH}{CLASS_NAME}/"
# 위 폴더는 개인환경에 맞추어 수정하시면 됩니다

os.makedirs("./models",exist_ok=True)
os.makedirs("./images",exist_ok=True)

BASE_SAVE_PATH = f"./models/{CLASS_NAME}/"
#pretrained 모델을 사용할때는 이미지 사이즈가 224,224가 맞습니다.
#300*300으로 해도 잘 나와서 추후 수정하지 않고 유지 하였으나 224,224로 사용하시는것이 일반적인방법입니다.
IMG_SIZE = (300, 300)
BATCH_SIZE = 16

# 10번 이하는 쏠림이 있는 모델이 생성되는 경우가 많음. 
EPOCHS = 1
# 학습이 안되는경우 learning rate를 조정하시는게 가장 효과적인 방법입니다.

LEARNING_RATE = 1.0e-6
SEED = 14



# Fixed seed

In [ ]:
# seed를 고정. 일정한 값 도출 및 추가되는 다른 것들에 대한 효과를 보기위해서 고정한다.
# 랜덤시드 제대로 고정됐는지 실험 완료
# 재현성을 위해 시드를 고정하지만 아래 3개 환경은 성능이 떨어지는 이슈가 있어서 고정했음
def set_seed(seed):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    # os.environ['TF_DETERMINISTIC_OPS'] = "1"
    # os.environ['TF_CUDNN_DETERMINISM'] = "1"
    # os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(SEED)

# Plot option

In [ ]:
#-------------------------------------------------------------------------------------------------------------* 
# AUC chart 옵션설정
def plot_roc(pred,y, foldnum):
    fpr, tpr, _ = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.savefig(f'./images/{CLASS_NAME}/AUC'+str(foldnum)+'.png')
    plt.show()

def plot_metrics(history, foldnum):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.savefig(f'./images/{CLASS_NAME}/accuracy_fold{foldnum}.png')
    plt.show()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.savefig(f'./images/{CLASS_NAME}/loss_fold{foldnum}.png')
    plt.show()

def plot_confusion_matrix(labels, predictions, foldnum):
    cm = confusion_matrix(labels, predictions)
    f, ax = plt.subplots(figsize=(4, 4))
    sns.heatmap(cm, annot=True, linewidths=0.01, cmap="Greens", linecolor="gray", fmt='.1f', ax=ax)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.savefig(f'./images/{CLASS_NAME}/confusion_matrix_fold{foldnum}.png')
    plt.show()
    return cm

# Callback

In [ ]:
#-------------------------------------------------------------------------------------------------------------* 
# Call back 옵션설정
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.90, patience=3, verbose=0, mode='auto', min_lr=1e-7)
checkpoint = ModelCheckpoint(f'{BASE_SAVE_PATH}/best.h5', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 0)
# early_stopping의 경우 30번 이하일 경우 모델성능이 매우 떨어지기 때문에 30 이상 결과를 위해해서 30으로 걸어둠.
early_stopping = EarlyStopping(monitor='val_loss', patience=30, mode='auto')

# 가중치 조절
- 불균형데이터로 실험할경우 사용하세요

In [ ]:
# 샘플 수
num_abnormal = 286
num_normal = 566
num_total = num_abnormal + num_normal

weight_normal = (1 / num_abnormal) * (num_total)/2.0
weight_abnormal = (1 / num_normal)  * (num_total)/2.0 

# 각 클래스의 가중치 계산
class_weights = {
    0: weight_abnormal,  # class 0 (abnormal)
    1: weight_normal   # class 1 (normal)
}

# 정규화를 위해 전체 가중치의 합으로 나눔
total = sum(class_weights.values())
class_weights = {k: v / total for k, v in class_weights.items()}

# 클래스 가중치 출력
print("Class weights: ", class_weights)


# Model Setting

In [ ]:
def build_vgg16_model():
    # 기본적으로 pretrained 모델은 224,224 를 사용합니다.
    # 기존 만들어진 이미지가 300*300 이기 때문에 300*300을 사용했지만 224*224를 권장합니다.
    conv_base = VGG16(include_top=False, input_shape=(300, 300, 3), weights="imagenet")
    for i, layer in enumerate(conv_base.layers):
        layer.trainable=False
    for layer in conv_base.layers[-20:]:
        layer.trainable = True
    #for i, layer in enumerate(conv_base.layers):
        #print(i, layer.name, layer.trainable)

    #-----------------------------------------------------------------------------------------------* 
    #vgg16모델 FineTunning           
    x = conv_base.output
    # flatten 사용시 376mb, flatten 대신 glovalAverage 사용하는것도 추천. (모델 용량이 많이 줄어듭니다.)
    #x = Flatten()(x) # Flatten dimensions to for use in FC layers
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x) # Dropout layer to reduce overfitting
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.4)(x) # Dropout layer to reduce overfitting #선택사항        
    x = Dense(256, activation='relu')(x) # 2단계에서 여기서 추출
    # sigmoid와 softmax중에 softmax를 채택한 이유는 이후 모델이 커지면 sigmoid는 수렴이 잘 안되는 경우가 발생했습니다.
    # softmax는 카테고리컬을 꼭 사용하세요 binary로 분류도 가능하지만 정확한 계산을 위해서 categorical을 사용하는것이 필수입니다.
    x = Dense(2, activation='softmax')(x) 
    # transfer_model.summary()
    return Model(inputs=conv_base.input, outputs=x)

In [ ]:
def train_model(train_dir, test_dir, foldnum, qnum):
    train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)




    # flow_from_directory의 경우 폴더만 구성되어 있으면 바로 분류 모델 생성이 가능하다. 어떤 폴더가 0 번이 되는지 label의 확인 필요
    # 알파벳 순으로 라벨이 결정되기 때문에 ex) mci_ad 의 경우 ad가 0번 label을 가지고 mci가 1번 라벨을 가짐.
    train_gen = train_datagen.flow_from_directory(
        train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=True, seed=SEED, subset="training")
    val_gen = train_datagen.flow_from_directory(
        train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=True, seed=SEED, subset="validation")
    test_gen = test_datagen.flow_from_directory(
        test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, shuffle=False, seed=SEED)

    model = build_vgg16_model()
    model.compile(optimizer=tf.optimizers.Nadam(learning_rate=LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(
        train_gen,
        steps_per_epoch=len(train_gen),
        epochs=EPOCHS,
        validation_data=val_gen,
        validation_steps=len(val_gen),
        verbose=1,
        callbacks=[lr_reduce, early_stopping,checkpoint]
    )

    val_loss = history.history['val_loss']
    val_accuracy = history.history['val_accuracy']

    # Confusion Matrix and Metrics
    predictions = model.predict(test_gen, steps=len(test_gen), verbose=0)
    pred_labels = np.argmax(predictions, axis=1)
    cm = plot_confusion_matrix(test_gen.labels, pred_labels, foldnum)

    tn, fp, fn, tp = cm.ravel()
    accuracy = (tn + tp) / (tn + tp + fn + fp)
    precision = precision_score(test_gen.labels, pred_labels, average='binary')
    recall = recall_score(test_gen.labels, pred_labels, average='binary')

    # Save model
    acc = round(accuracy * 100, 1)
    model_save_path = f"{BASE_SAVE_PATH}/model_{CLASS_NAME}_q{qnum}_fold{foldnum}_acc{acc}.h5"
    model.save(model_save_path)


    # Plot metrics
    plot_metrics(history, foldnum)
    plot_roc(pred_labels,test_gen.labels, foldnum)
    print(f"Precision: {precision*100:.2f}%")
    print(f"Recall: {recall*100:.2f}%")
    print(f"Accuracy: {accuracy*100:.2f}%")
    
    return model

In [ ]:
# Main loop
for qnum in range(1, 12):
    for foldnum in range(1, 6):
        base_dir = f"{private_dir}{qnum}/iterations"
        train_dir = os.path.join(base_dir, f'iteration{foldnum}/train/')
        test_dir = os.path.join(base_dir, f'iteration{foldnum}/test/')

        print(f"Training fold {foldnum} for question {qnum}...")
        model=train_model(train_dir, test_dir, foldnum, qnum)
        K.clear_session()

# 틀린것 확인할때 아래 사용

In [ ]:

def visualize_wrong_predictions(model, test_generator):
    predicted_result = model.predict(test_generator)
    predicted_labels = np.argmax(predicted_result, axis=1)

    test_labels = test_generator.labels

    wrong_result = []

    for n in range(0, len(test_labels)):
        if predicted_labels[n] != test_labels[n]:
            wrong_result.append(n)

    samples = random.choices(population=wrong_result, k=16)

    count = 0
    nrows = ncols = 4

    plt.figure(figsize=(12,8))

    for n in samples:
        count += 1
        plt.subplot(nrows, ncols, count)
        plt.imshow(test_generator[n].reshape(28, 28), cmap='Greys', interpolation='nearest')
        tmp = "Label:" + str(test_labels[n]) + ", Prediction:" + str(predicted_labels[n])
        plt.title(tmp)

    plt.tight_layout()
    plt.show()